In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools
import pickle

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20
T = 20
#cpptools.setup_nlopt(folder='cppfuncs/', do_print = True) #install nlopt



#samme antal observationer: KØR WAGE TYPE; LOVEKONTROL; ETC
#norm1 N = 9,000    model FC  
#norm1 N = 2,400    model NC 
#norm1 N = 2,100    model LC 




#NEW 9000
#model LC 2700      6000
#model NC 3000      6400
#model FC 9200      21000


# restr feas 20000 (prøv at kør det med 9000 bagefter) 

#rest feas 9000
#model_FC 9200 --> 6200
#model LC 2700  --> 6800
#model NC 3000 --> 7800  

#rest nonfeas 9000
#model_FC 9200 -->  6200
#model LC 2700  -->  489... kan ikke køre
#model_LC 12000 -->  2400
#model NC 3000 --> 6800  


#Jeg skal køre både NC og FC IGEN; JEG ER I TVIL OM HVILKEN EN AF DEM SOM JEG HAR KØRT

#Kør FC først og se om den er helt ens med NC? Jeg er i tvivl da jeg kørte NC om jeg kom til at køre FC 


# Load the model

In [ ]:
name = 'model_FC'
model_name = "model_FC"
do_distr_yerror = "none"
do_yerror = 0
#do_BMI_dummy = False 
do_sep_shock = False 
#name_error = f'{do_distr_yerror}{do_yerror}std_Yerror_rest0_restOutl'
name_error = f'{do_distr_yerror}{do_yerror}std_Yerror_restr_nonfeas'
N = 9200
NN = "NN" #NN er samme antal observationer, N er samme antal simuleringer
NNO = 9000
rest_sample_feas = True
rest_sample_nonfeas    = True
#name_error = "uni2std_Yerror"

specs = {}
#specs.update({f'test2':{'latexname':'limited', 'par':{ 'T':T, 'num_K': 5, 'num_A': 10 ,  'do_HK': False, 'threads':threads,'bargaining':0}}})


#specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':1}}})
#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':2}}})
specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'simN': N,  'threads':threads,'bargaining':0}}})

# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    
    # setup model
    model = LimitedCommitmentModelClass(name=name,par=spec['par'])
    model.spec = spec

    compile_now = True if do_compile & (m==0) else False
    model.link_to_cpp(force_compile=compile_now)
    

# setup model
#model = LimitedCommitmentModelClass(name=name)    
#model.link_to_cpp(force_compile=True)


#load par and sol
pickle_file = f'output/{name}_sol.pickle'

with open(pickle_file, 'rb') as f:
    model.sol = pickle.load(f)

pickle_file = f'output/{name}_par.pickle'

with open(pickle_file, 'rb') as f:
    model.par = pickle.load(f)
    model.par.simN = N

In [ ]:
#Do 1 time estimate
data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror )

data = Estimate.create_variable(data, model.par)
if rest_sample_nonfeas:  
    if model_name != "model_FC":
        # Remove those who do not update bargaining weight
        data = data[data['delta_log_barganing'] != 0]
        
        #only small shocks to bargining weight
        data = data[data['delta_log_barganing'] < 0.1]
        data = data[data['delta_log_barganing'] > - 0.1]


if rest_sample_feas:  
    #not at the limit
    
    data = data[data['hours_w'] != 1]
    data = data[data['hours_m'] != 1]
    
    data = data[data['hours_w_l'] != 1]
    data = data[data['hours_m_l'] != 1]
    
    data = data[data['hours_w_l2'] != 1]
    data = data[data['hours_m_l2'] != 1]


    
    data = data[data['delta_hours_w'] < 0.1]
    data = data[data['delta_hours_m'] < 0.1]
    data = data[data['delta_hours_w'] > - 0.1]
    data = data[data['delta_hours_m'] > -0.1]
data = Estimate.aux_est(data,model.par)
#print(data)


_, wald_FC_w , wald_NC_w, _,_,  _ = Estimate.main_est(data,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=2, do_control_love = False, part_earning_simple =1, control_cons = 1, wealth_love = 1, do_sep_shock=False)


In [ ]:
#KØRE ALLE
#BAGEFTER KØR FULDE MODEL MED TRUE OMEGA OG LOVE = TRUE
#run regression S times
list_wage = ["est_omega","true_omega", "wage"]
list_wage = ["est_omega","true_omega"]
list_love = [False, True]

#list_wage = ["est_omega",]
#list_love = [False,]
#list_wage = ["est_omega","true_omega",]
#list_love = [True,]
#name_est = "No_love_est_omega"
model.par.seed = 2024
np.random.seed(model.par.seed)

no_sim = 100
wald_FC_w_pval = np.nan + np.ones((4,2,2,len(list_wage),len(list_love),no_sim) )
wald_NC_w_pval = np.nan + np.ones((4,2,2,len(list_wage),len(list_love),no_sim) )
wald_FC_m_pval = np.nan + np.ones((4,2,2,len(list_wage),len(list_love),no_sim) )
wald_NC_m_pval = np.nan + np.ones((4,2,2,len(list_wage),len(list_love),no_sim) )


#wald_FC_w_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) ) 
#wald_NC_w_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) )
#wald_FC_m_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) )
#wald_NC_m_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) )


coef_w = {}
coef_m = {}





N_w= np.nan + np.ones(no_sim) 
N_m= np.nan + np.ones(no_sim) 
#loop over shadow values
#for j in range(4):
#    j = j +1
for j in [1,2,3]:
    #jeg dropper nummer 4
    for jj in [1,2]:
        jjj = 1
        list_p = [False, True]
        list_wage = ["est_omega","true_omega"]
        list_love = [False, True]
        if j == 1:
            list_p = [False,]
            list_wage = ["est_omega",]
            list_love = [False,]
        for p in list_p:
            for im, do_love in enumerate(list_love):
                for ik, do_wage in enumerate(list_wage):
                    do_sep_shocks = False
                    for i in range(no_sim):
                        data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror)
                        data = Estimate.create_variable(data, model.par)
                        if rest_sample_nonfeas:  
                            if model_name != "model_FC":
                                # Remove those who do not update bargaining weight
                                data = data[data['delta_log_barganing'] != 0]
                                
                                #only small shocks to bargining weight
                                data = data[data['delta_log_barganing'] < 0.1]
                                data = data[data['delta_log_barganing'] > - 0.1]


                        if rest_sample_feas:  
                            #not at the limit
                            data = data[data['hours_w'] != 1]
                            data = data[data['hours_m'] != 1]
                            #only small schock to hours
                            data = data[data['delta_hours_w'] < 0.1]
                            data = data[data['delta_hours_m'] < 0.1]
                            data = data[data['delta_hours_w'] > - 0.1]
                            data = data[data['delta_hours_m'] > -0.1]
                        #if k:
                        data = Estimate.aux_est(data,model.par)
                        _, wald_FC_w , wald_NC_w, wald_FC_noW_w , wald_NC_noW_w, c_w = Estimate.main_est(data,gender= "w", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj, wealth_love=jjj, do_sep_shock = do_sep_shocks, do_true_barg = p)
                        _, wald_FC_m , wald_NC_m, wald_FC_noW_m , wald_NC_noW_m, c_m = Estimate.main_est(data,gender= "m", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj, wealth_love=jjj, do_sep_shock = do_sep_shocks, do_true_barg = p)
                        ip = int(p)
                        wald_FC_w_pval[j-1, jj-1,ip,ik, im, i] = wald_FC_w.pvalue
                        wald_FC_m_pval[j-1, jj-1,ip,ik, im, i] = wald_FC_m.pvalue
                        wald_NC_w_pval[j-1, jj-1,ip,ik, im, i] = wald_NC_w.pvalue
                        wald_NC_m_pval[j-1, jj-1,ip,ik, im, i] = wald_NC_m.pvalue

                        #wald_FC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_w.pvalue
                        #wald_FC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_m.pvalue
                        #wald_NC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_w.pvalue
                        #wald_NC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_m.pvalue
                        
                        coef_w[f'shadow{j}_inc{jj}_cons{jj}_WL{jjj}_wage{ik}_love{im}_barg{ip}_sim{i}'] =  c_w
                        coef_m[f'shadow{j}_inc{jj}_cons{jj}_WL{jjj}_wage{ik}_love{im}_barg{ip}_sim{i}'] =  c_m
                        
            


In [ ]:
#SAVE PICKLE
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_m_pval,f)
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_m_pval,f)

pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_m,f)

pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_w,f)

In [ ]:
stop
#KØRE ALLE

#BAGEFTER KØR FULDE MODEL MED TRUE OMEGA OG LOVE = TRUE
#run regression S times
#list_wage = ["est_omega","true_omega", "wage"]
#list_love = [False, True]

list_wage = ["est_omega",]
list_love = [False,]
#list_wage = ["est_omega","true_omega",]
#list_love = [True,]
#name_est = "No_love_est_omega"

np.random.seed(model.par.seed)

no_sim = 100
wald_FC_w_pval = np.nan + np.ones((7,3,2,len(list_wage),len(list_love),no_sim) )
wald_NC_w_pval = np.nan + np.ones((7,3,2,len(list_wage),len(list_love),no_sim) )
wald_FC_m_pval = np.nan + np.ones((7,3,2,len(list_wage),len(list_love),no_sim) )
wald_NC_m_pval = np.nan + np.ones((7,3,2,len(list_wage),len(list_love),no_sim) )


#wald_FC_w_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) ) 
#wald_NC_w_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) )
#wald_FC_m_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) )
#wald_NC_m_fval = np.nan + np.ones((7,3,3,len(list_wage),len(list_love),no_sim) )


coef_w = {}
coef_m = {}



wald_FC_w_negative = np.nan + np.ones((7,3,2,len(list_wage),len(list_love))) 
wald_NC_w_negative = np.nan + np.ones((7,3,2,len(list_wage),len(list_love)))
wald_FC_m_negative = np.nan + np.ones((7,3,2,len(list_wage),len(list_love)))
wald_NC_m_negative = np.nan + np.ones((7,3,2,len(list_wage),len(list_love)))





N_w= np.nan + np.ones(no_sim) 
N_m= np.nan + np.ones(no_sim) 
#loop over shadow values
#for j in range(4):
#    j = j +1
for j in [1,2,3]:   #shadow value
    for jj in [1,2,3]:    #income & cons     
            list2 = [1,2]
            if jj == 1:
                 list2 = [1,]
            for jjj in list2:   #wealth (and love)
        

                #do_wage = "est_omega"
                for ik, do_wage in enumerate(list_wage):
                # # Her skal jeg have 3 strings
                    for im, do_love in enumerate(list_love):
                    #her skal jeg evt loope over love
                    
                        for i in range(no_sim):
                            data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror)
                            data = Estimate.create_variable(data, model.par)
                            if rest_sample:                
                                data = data[data['delta_log_barganing']!= 0]
                            #if k:
                            data = Estimate.aux_est(data,model.par)
                            _, wald_FC_w , wald_NC_w, wald_FC_noW_w , wald_NC_noW_w, c_w = Estimate.main_est(data,gender= "w", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj, wealth_love=jjj, BMI_dummy = do_BMI_dummy)
                            _, wald_FC_m , wald_NC_m, wald_FC_noW_m , wald_NC_noW_m, c_m = Estimate.main_est(data,gender= "m", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj, wealth_love=jjj,BMI_dummy = do_BMI_dummy)
                    
                            wald_FC_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_w.pvalue
                            wald_FC_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_m.pvalue
                            wald_NC_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_w.pvalue
                            wald_NC_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_m.pvalue

                            #wald_FC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_w.pvalue
                            #wald_FC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_m.pvalue
                            #wald_NC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_w.pvalue
                            #wald_NC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_m.pvalue
                            
                            coef_w[f'shadow{j}_inc{jj}_cons{jj}_WL{jjj}_wage{ik}_love{im}_sim{i}'] =  c_w
                            coef_m[f'shadow{j}_inc{jj}_cons{jj}_WL{jjj}_wage{ik}_love{im}_sim{i}'] =  c_m
                                
                        wald_FC_w_negative[j-1,jj-1, jjj-1, ik,im] = np.sum(wald_FC_w_pval[j-1,ik,im,:]< 0.05)/no_sim
                        wald_FC_m_negative[j-1,jj-1, jjj-1, ik,im] = np.sum(wald_FC_m_pval[j-1,ik,im:]< 0.05)/no_sim
                        wald_NC_w_negative[j-1,jj-1, jjj-1, ik,im] = np.sum(wald_NC_w_pval[j-1,ik,im:]< 0.05)/no_sim
                        wald_NC_m_negative[j-1,jj-1, jjj-1, ik,im] = np.sum(wald_NC_m_pval[j-1,ik,im,:]< 0.05)/no_sim

                    #wald_FC_w_noW_negative[j-1,ik,im] = np.sum(wald_FC_noW_w_pval[j-1,ik,im,:]< 0.05)/no_sim
                    #wald_FC_m_noW_negative[j-1,ik,im] = np.sum(wald_FC_noW_m_pval[j-1,ik,im,:]< 0.05)/no_sim
                    #wald_NC_w_noW_negative[j-1,ik,im] = np.sum(wald_NC_noW_w_pval[j-1,ik,im,:]< 0.05)/no_sim
                    #wald_NC_m_noW_negative[j-1,ik,im] = np.sum(wald_NC_noW_m_pval[j-1,ik,im,:]< 0.05)/no_sim



In [ ]:
#SAVE PICKLE
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_m_pval,f)
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_m_pval,f)

pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_m,f)

pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_w,f)

In [ ]:
stop

In [ ]:
print(coef_w)

In [ ]:
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'

with open(pickle_file, 'rb') as f:
    test = pickle.load(f)
print(test)

In [ ]:
#LOAD PICKLE (SEPERETAE FILE )

pickle_file = f'output/TRUE{name}_w_N{model.par.simN}_S{no_sim}_{do_distr_yerror}{do_yerror}_testFC.pickle'


with open(pickle_file, 'rb') as f:
    FC_woman = pickle.load(f)
print(FC_woman)

In [ ]:
#data[['wage_m','wage_w']].describe()


#LAV FORDELING MED MAIN OG NONP , og med MAIN og xx

#wage_w      =  np.exp(model.par.wage_const_w +model.par.wage_K_w*model.sim.Kw)   
#y_w         =  wage_w*model.sim.labor_w
#wage_m      =  np.exp(model.par.wage_const_m+model.par.wage_K_m* model.sim.Km)
#y_m         =  wage_m*model.sim.labor_m

In [ ]:
stop

In [ ]:

#load par and sol
pickle_file = f'output/{name}_sol.pickle'

with open(pickle_file, 'rb') as f:
    model.sol = pickle.load(f)


In [ ]:
#TEST 
data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2)
print(np.mean(data['hours_w'])-np.mean(data['hours_m']))
print(np.mean(data['wage_w'])-np.mean(data['wage_m']))
print(np.mean(data['earnings_w'])-np.mean(data['earnings_m']))
print(np.mean(data['BMI_w'])-np.mean(data['BMI_m']))
data = Estimate.create_variable(data, model.par)


In [ ]:
#data[['wage_w_orig','wage_m_orig', 'hours_w_orig','hours_m_orig','wealth_orig', 'wage_w','wage_m', 'hours_w','hours_m','wealth', 'cons']].describe()

#wage_w_orig = np.exp(model.par.wage_const_w +model.par.wage_K_w*model.sim.Kw) 
#wage_m_orig = np.exp(model.par.wage_const_m +model.par.wage_K_m*model.sim.Km) 

#std = np.std(wage_w_orig)
#std2 = np.std(wage_m_orig)
#print(std)
#print(std2)
iK= 6
iZ = 0

test = np.allclose(model.sol.Vw_single, model.sol.Vm_single)
print(f'V_single {test}')
test = np.allclose(model.sol.labor_w_single , model.sol.labor_m_single)
print(f'labor_single {test}')
test = np.allclose(model.sol.cons_w_single , model.sol.cons_m_single)
print(f'cons_single {test}')
test = np.allclose(model.sol.Vw_couple[:, iZ, iZ, 5, :, :, iK, iK], model.sol.Vm_couple[:, iZ, iZ, 5, :, :, iK, iK], atol = 0.1)
print(f'V_couple {test}')
test = np.allclose(model.sol.labor_w_couple[:, iZ, iZ, 5, :, :, iK, iK] , model.sol.labor_m_couple[:, iZ, iZ, 5, :, :, iK, iK], atol = 0.1 )
print(f'labor couple {test}')
test = np.allclose(model.sol.cons_w_couple[:, iZ, iZ, 5, :, :, iK, iK], model.sol.cons_m_couple[:, iZ, iZ, 5, :, :, iK, iK], atol = 0.1)
print(f'C couple {test}')
test = np.allclose(model.sol.Vw_remain_couple[:,iZ, iZ, 5, :, :, iK, iK], model.sol.Vm_remain_couple[:, iZ, iZ,  5, :, :, iK, iK], atol = 0.1)
print(f'V_remain {test}')
test = np.allclose(model.sol.labor_w_remain_couple[:, iZ, iZ, 5, :, :, iK, iK] , model.sol.labor_m_remain_couple[:, iZ, iZ, 5, :, :, iK, iK], atol = 0.1)
print(f'labor_remain {test}')
test = np.allclose(model.sol.cons_w_remain_couple[:, iZ, iZ, 5, :, :, iK, iK], model.sol.cons_m_remain_couple[:, iZ, iZ, 5, :, :, iK, iK], atol = 0.1)
print(f'cons_remain {test}')
test = np.isclose(model.sol.labor_w_remain_couple[:, iZ, iZ, 5, :, :, iK, iK] , model.sol.labor_m_remain_couple[:,iZ, iZ,  5, :, :, iK, iK], atol = 0.1)
ind = np.where(np.logical_not(test))
print(ind) #all difference is where wealth high


ind = np.where(np.logical_not(test))
print(ind)
np.random.seed(model.par.seed)
model.simulate()

In [ ]:
print(np.mean(model.sim.labor_w )-np.mean(model.sim.labor_m ))
print(np.mean(model.sim.Kw )-np.mean(model.sim.Km ))
print(np.mean(model.sim.Zw )-np.mean(model.sim.Zm ))
print(np.mean(model.sim.cons_w )-np.mean(model.sim.cons_m ))
print(np.mean(model.sim.exp_w )-np.mean(model.sim.exp_m )) 
print(np.mean(model.sim.Aw )-np.mean(model.sim.Am )) 

In [ ]:
print(np.mean(data['y_w'])-np.mean(data['y_m']))
print(np.mean(data['log_earnings_w'])-np.mean(data['log_earnings_m']))
print(np.mean(data['delta_log_wage_w'])-np.mean(data['delta_log_wage_m']))
print(np.mean(data['omega_res_w'])-np.mean(data['omega_res_m']))
print(np.mean(data['delta_omega_w'])-np.mean(data['delta_omega_m']))
print(np.mean(data['control_part_inc_w'])-np.mean(data['control_part_inc_m']))
print(np.mean(data['delta_log_BMI_w'])-np.mean(data['delta_log_BMI_m']))


print(np.mean(data['log_earnings_w_l'])-np.mean(data['log_earnings_m_l']))
print(np.mean(data['omega_res_w_l'])-np.mean(data['omega_res_m_l']))
print(np.mean(data['delta_omega_w_l'])-np.mean(data['delta_omega_m_l']))
print(np.mean(data['delta_log_BMI_w_l'])-np.mean(data['delta_log_BMI_m_l']))


print(np.mean(data['omega_res_w_l2'])-np.mean(data['omega_res_m_l2']))
print(np.mean(data['delta_omega_w_l2'])-np.mean(data['delta_omega_m_l2']))
print(np.mean(data['delta_log_BMI_w_l2'])-np.mean(data['delta_log_BMI_m_l2']))
